<a href="https://colab.research.google.com/github/padolekrushna/ML_exercise.pynb/blob/main/ML_Algo_Practice_home_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BANGALORE HOUSE PRICE PREDICTION MODEL

### IMPORT LIBRARIES

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

### LOAD DATASET

In [33]:
df1 = pd.read_csv("/content/banglore_home_prices_model.pickle")
df1.head()

EmptyDataError: No columns to parse from file

### EXPLORATORY DATA ANALYSIS

In [4]:
df1.shape

NameError: name 'df1' is not defined

In [ ]:
df1.columns

In [ ]:
df1['area_type'].unique()

In [ ]:
df1['area_type'].value_counts()

**NOTE:** DROP UNNECESSARY FEATURES

In [5]:
df2 = df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.shape

NameError: name 'df1' is not defined

### DATA CLEANING

In [6]:
df2.isnull().sum()

NameError: name 'df2' is not defined

In [ ]:
df2.shape

In [ ]:
df3 = df2.dropna()
df3.isnull().sum()

In [ ]:
df3.shape

### FEATURE ENGINEERING

In [7]:
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
df3.bhk.unique()

NameError: name 'df3' is not defined

In [ ]:
df3[df3['total_sqft'].apply(is_float)].head(10)

In [8]:
df3.head(3)

NameError: name 'df3' is not defined

In [ ]:
df3['total_sqft']


**ABOVE DATA SHOWS THAT TOTAL SQFT CAN BE A RANGE (E.G. 2100-2850). FOR SUCH CASES WE CAN JUST TAKE AVERAGE OF MIN & MAX VALUE IN THE RANGE. THERE ARE OTHER CASES WHERE VALUES ARE IN SQM WHICH CAN BE CONVERTED TO SQFT USING UNIT CONVERSION.**

###  1 Square foot is equal to 0.0023 Decimal
## Decimal = Square foot x 0.0023
##Usage:
If you input a string like "1000-1200", it will return 1100.0.                  
If you input "900", it will return 900.0.                                       
If you input an invalid string, it will return None.

In [9]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [10]:
df4 = df3.copy()
df4.total_sqft = df4.total_sqft.apply(convert_sqft_to_num)
df4 = df4[df4.total_sqft.notnull()]
df4.head(2)

NameError: name 'df3' is not defined

**FOR ROW BELOW, IT SHOWS TOTAL SQFT AS 2475 WHICH IS AN AVERAGE OF THE RANGE 2100-2850**

In [ ]:
df4.loc[30]

In [11]:
(2100+2850)/2

2475.0

**ADD NEW FEATURE CALLED PRICE PER SQUARE FEET**

price_per_sqft= price×100,000/total_sqft

multiplying by 100,000 adjusts the value to match the scale of the square footage

In [12]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()

NameError: name 'df4' is not defined

In [13]:
df5_stats = df5['price_per_sqft'].describe()
df5_stats

NameError: name 'df5' is not defined

In [ ]:
df5.to_csv("bhp.csv",index=False)
df5.head()

**EXAMINE LOCATIONS WHICH IS A CATEGORICAL VARIABLE. WE NEED TO APPLY THE DIMENSIONALITY REDUCTION TECHNIQUE HERE TO REDUCE THE NUMBER OF LOCATIONS**

In [ ]:
df5.location = df5.location.apply(lambda x: x.strip())
location_stats = df5['location'].value_counts(ascending=False)
location_stats

In [14]:
location_stats.values.sum()

NameError: name 'location_stats' is not defined

In [ ]:
len(location_stats[location_stats>10])

In [ ]:
len(location_stats)

In [15]:
len(location_stats[location_stats<=10])

NameError: name 'location_stats' is not defined

### DIMENSIONALITY REDUCTIONS

**ANY LOCATION HAVING LESS THAN 10 DATA PINTS SHOULD BE TAGGED AS "OTHER" LOCATION. THIS WAY NUMBER OF CATEGORIES CAN BE REDUCED BY HUGE AMOUNT. LATER ON WHEN WE DO ONE HOT ENCODING, IT WILL HELP US WITH HAVING FEWER DUMMY COLUMNS.**

In [16]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

NameError: name 'location_stats' is not defined

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

In [17]:
df5.head(10)

NameError: name 'df5' is not defined

### OUTLIER REMOVAL USING BUSINESS LOGIC

**AS A DATA SCIENTIST S A DATA SCIENTIST WHEN YOU HAVE A CONVERSATION WITH YOUR BUSINESS MANAGER (WHO HAS EXPERTISE IN REAL ESTATE), HE WILL TELL YOU THAT NORMALLY SQUARE FT PER BEDROOM IS 300 (I.E. 2 BHK APARTMENT IS MINIMUM 600 SQFT. IF YOU HAVE FOR EXAMPLE 400 SQFT APARTMENT WITH 2 BHK THAN THAT SEEMS SUSPICIOUS AND CAN BE REMOVED AS AN OUTLIER. WE WILL REMOVE SUCH OUTLIERS BY KEEPING OUR MINIMUM THRESOLD PER BHK TO BE 300 SQFT**

In [18]:
df5[df5.total_sqft/df5.bhk<300].head()

NameError: name 'df5' is not defined

**CHECK THE ABOVE DATA POINTS. WE HAVE 6 BHK APARTMENTS WITH 1020 SQFT. ANOTHER ONE IS 8 BHK AND THE TOTAL SQFT IS 600. THESE ARE CLEAR DATA ERRORS THAT CAN BE REMOVED SAFELY**

In [19]:
df5.shape

NameError: name 'df5' is not defined

In [ ]:
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

### OUTLIER REMOVAL USING STANDARD DEVIATION AND MEAN

In [20]:
df6.price_per_sqft.describe()

NameError: name 'df6' is not defined

**HERE WE FIND THAT MIN PRICE PER SQFT IS 267 RS/SQFT WHEREAS MAX IS 12000000, THIS SHOWS A WIDE VARIATION IN PROPERTY PRICES. WE SHOULD REMOVE OUTLIERS PER LOCATION USING MEAN AND ONE STANDARD DEVIATION**

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7 = remove_pps_outliers(df6)
df7.shape

**LET'S CHECK IF FOR A GIVEN LOCATION HOW DOES THE 2 BHK AND 3 BHK PROPERTY PRICES LOOK LIKE**

In [21]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+', color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()

plot_scatter_chart(df7,"Rajaji Nagar")

NameError: name 'df7' is not defined

In [ ]:
plot_scatter_chart(df7,"Hebbal")

In [22]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)
df8 = df7.copy()
df8.shape

NameError: name 'df7' is not defined

**PLOT SAME SCATTER CHART AGAIN TO VISUALIZE PRICE_PER_SQFT FOR 2 BHK AND 3 BHK PROPERTIES**

In [ ]:
plot_scatter_chart(df8,"Rajaji Nagar")

In [23]:
plot_scatter_chart(df8,"Hebbal")

NameError: name 'df8' is not defined

**BASED ON ABOVE CHARTS WE CAN SEE THAT DATA POINTS HIGHLIGHTED IN RED BELOW ARE OUTLIERS AND THEY ARE BEING REMOVED DUE TO REMOVE_BHK_OUTLIERS FUNCTION**

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

### OUTLIER REMOVAL USING BATHROOMS FEATURE

In [24]:
df8.bath.unique()

NameError: name 'df8' is not defined

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

In [25]:
df8[df8.bath>10]

NameError: name 'df8' is not defined

**IT IS UNUSUAL TO HAVE 2 MORE BATHROOMS THAN NUMBER OF BEDROOMS IN A HOME**

In [ ]:
df8[df8.bath>df8.bhk+2]

**AGAIN THE BUSINESS MANAGER HAS A CONVERSATION WITH YOU (I.E. A DATA SCIENTIST) THAT IF YOU HAVE A 4 BEDROOM HOME AND EVEN IF YOU HAVE A BATHROOM IN ALL 4 ROOMS PLUS ONE GUEST BATHROOM, YOU WILL HAVE A TOTAL BATH = TOTAL BED + 1 MAX. ANYTHING ABOVE THAT IS AN OUTLIER OR A DATA ERROR AND CAN BE REMOVED**

In [26]:
df9 = df8[df8.bath<df8.bhk+2]
df9.shape

NameError: name 'df8' is not defined

In [ ]:
df9.head(2)

In [27]:
df10 = df9.drop(['size','price_per_sqft'],axis='columns')
df10.head(3)
df10.location

NameError: name 'df9' is not defined

### USE ONE HOT ENCODING FOR LOCATION

In [ ]:
dummies = pd.get_dummies(df10.location)
dummies.head(10)

In [28]:
df11 = pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df11.head()

NameError: name 'df10' is not defined

In [ ]:
df12 = df11.drop('location',axis='columns')
df12.head(2)

### BUILDING A MODEL

In [29]:
df12.shape

NameError: name 'df12' is not defined

In [ ]:
X = df12.drop(['price'],axis='columns')
X.head(3)

In [30]:
X.shape

NameError: name 'X' is not defined

In [ ]:
y = df12.price
y.head(3)

In [ ]:
len(y)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

NameError: name 'X' is not defined

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

### USE K FOLD CROSS VALIDATION TO MEASURE ACCURACY OF OUR LINEAR REGRESSION MODEL

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

**WE CAN SEE THAT IN 5 ITERATIONS WE GET A SCORE ABOVE 80% ALL THE TIME. THIS IS PRETTY GOOD BUT WE WANT TO TEST FEW OTHER ALGORITHMS FOR REGRESSION TO SEE IF WE CAN GET AN EVEN BETTER SCORE. WE WILL USE GRIDSEARCHCV FOR THIS PURPOSE**

In [32]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

NameError: name 'lr_clf' is not defined

### EXPORT LOCATION AND COLUMN INFORMATION TO A FILE THAT WILL BE USEFUL LATER ON IN OUR PREDICTION APPLICATION

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))